In [ ]:
# pip install ipywidgets
# pip install joblib
# pip install nltk
# pip install pandas
# pip install pyaudio
# pip install pyttsx3
# pip install queue
# pip install scikit-learn
# pip install SpeechRecognition
# pip install sounddevice
# pip install librosa
# pip install matplotlib
# pip install scipy
# pip install vosk

In [1]:
import ipywidgets as widgets
import threading
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue()
recordings = Queue()

# widget buttons for the project is defined here..

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)   

clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='info',
    tooltip='Clear',
    icon='eraser'
)   

# setting up the output widget display area with a border.

output = widgets.Output(layout={'border': '2px solid black'})

# When clicked on "Record" button, widget will start multiple thread to record microphone and transcribe the recorded audio.

def start_recording(data):
    output.clear_output() 
    messages.put(True)

    with output:
        display("Started Recording...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

# Once "Stop" is clicked, all thread will be switched off and recording will be stopped.

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped Recording.")

# This is to clear the output widget.

def clear_display_area(data):
    output.clear_output()

   
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)
clear_button.on_click(clear_display_area)

display(record_button, stop_button, output, clear_button)   

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output(layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right='2px solid b…

Button(button_style='info', description='Clear', icon='eraser', style=ButtonStyle(), tooltip='Clear')

In [2]:
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-en-us-0.22")

In [3]:
import pyttsx3

def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [4]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import os
import joblib
import pickle
import warnings

warnings.filterwarnings('ignore')

# Defining port stemmer 
porter = PorterStemmer()

# Get the current working directory (where the script is being executed)
current_dir = os.getcwd()

# Loading the tfidf vectorizer
vectorizer_path = os.path.join(current_dir, "tfidf_vectorizer.pkl")
with open(vectorizer_path, 'rb') as f:
    Tfidf_Vectorizer = pickle.load(f)

# Loading the Model
model_path = os.path.join(current_dir, "Threat_identifier_SVM.joblib")
TI_model = joblib.load(model_path)


In [5]:
import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

for i in range(0, numdevices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Headset Microphone (Jabra Link 
Input Device id  2  -  Digital Microphone (Cirrus Logi


In [6]:
CHANNELS = 1
FRAME_RATE = 44100
chunk=512
RECORD_SECONDS = 6
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=512):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=1,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [7]:
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

@output.capture()
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        output.append_stdout(text + '\n')
        prediction = predict_sentiment(text)
        if((prediction == 1)):
            msg = "This is VIOLENT or OFFENSIVE : I am making EMERGENCY call..."
            speak_text(msg)
            output.append_stdout('\t' + msg + '\n')
        else:
            msg = "I see this as a NORMAL conversation, I am NOT making emergency calls..."
            speak_text(msg)
            output.append_stdout('\t' + msg + '\n')
            

In [8]:

def predict_sentiment(sentence):
    # Define the cleaning function
    def clean_and_stem_text(text):
        text = re.sub(r'\n', ' ', text)  # Replace line breaks with space
        text = re.sub(r'\d+', '', text)  # Remove digits
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = text.lower()  # Convert to lowercase
        words = text.split()  # Split text into words
        stemmed_words = [porter.stem(word) for word in words]  # Stem each word
        return ' '.join(stemmed_words)
    
    # Define the cleaning pipeline
    cleaned_sentence = clean_and_stem_text(sentence)
    
    # Convert the cleaned sentence into a Series
    sentence_series = pd.Series(cleaned_sentence)
    
    # Convert the cleaned sentence into TF-IDF vector format
    sentence_tfidf = Tfidf_Vectorizer.transform(sentence_series)
    
    # Make prediction using the classifier
    prediction = TI_model.predict(sentence_tfidf)
    
    return prediction